# Sensitivity analysis using Sobol indices

In [1]:
import numpy as np
from model import NeoHookeanSolutionGenerator
from matplotlib import pyplot as plt

import seaborn as sns
%matplotlib inline

sns.set_theme(style="white")

from matplotlib.colors import to_rgb
import pandas as pd
from scipy.stats import gaussian_kde

from tqdm.notebook import tqdm as tqdm
from itertools import product

## Common setup

In [2]:
parameter_keys = ["c_1", "V_wall", "nu_s"]

In [3]:
# These are fixed, see synthetic_data
omega_val = np.pi
L_val = 2.0
zeta_val = 0.5
nu_f_val = 0.125
# V_wall_val = 1.0

offset = 0.0
res_t = 20
# time stations
time_samples = np.linspace(
    0 + offset / omega_val, (2 * np.pi) / omega_val, res_t, endpoint=False
)
# number of data points
n_samples = len(time_samples)
# y stations
y_stations = np.array([0.2, 0.4, 0.6, 0.8])

In [4]:
# Solution generator
solution_generator = NeoHookeanSolutionGenerator(omega_val, L_val, zeta_val, nu_f_val)

### Model evaluation

In [5]:
def generate_signal(
    fluid_velocity_gen, solid_velocity_gen, T=time_samples, y_stations=y_stations
):
    y_profiles = np.zeros((len(y_stations)))

    for i_station, y_station in enumerate(y_stations):
        solid_velocity = solid_velocity_gen(y_station, T)
        fluid_velocity = fluid_velocity_gen(y_station, T)
        solid_mask = y_station < 0.5
        # Take the max over time as a scalar indicator
        signal = solid_mask * solid_velocity + (1.0 - solid_mask) * fluid_velocity
        y_profiles[i_station] = 0.5 * (np.max(signal) - np.min(signal))

    return y_profiles

def signal_from_(estimates):
    global solution_generator
    v_f, v_s = solution_generator.generate_velocities_for(*estimates)
    signal = generate_signal(v_f, v_s)
    return signal

## Chains from Bayesian inference

In [6]:
chains = pd.read_pickle("chains.pkl")

## Kernel Density (pdf) Estimates

In [7]:
kdes = {}
limits = {'c_1': (0.01, 0.12), 'V_wall' : (0.9, 1.1), 'nu_s' : (0.0, 0.035)}

for parameter, chain in chains.items():
    kdes[parameter] = gaussian_kde(chain)
    # x = np.linspace(np.min(chain), np.max(chain), 100)
    # p = 75 * f(x)
    val = kdes[parameter].integrate_box(limits[parameter][0], limits[parameter][1])
    print("Integrated pdf is ", val)
    
kdes['nu_s'].integrate_box(0.0, 1.0)

Integrated pdf is  0.9993386388823118
Integrated pdf is  0.9999998855162321
Integrated pdf is  0.9999976534691107


0.999997653513043

# Sobol index calculations

## Setup

In [8]:
n_quadrature_points = [21, 21, 41]
n_quadrature_points = dict(zip(parameter_keys, n_quadrature_points))

quadrature_points = {}
delta_points = {}
quadrature_weights = {}
numerical_pdfs = {}
quadrature_pdfs = {}

limits = {'c_1': (0.01, 0.12), 'V_wall' : (0.9, 1.1), 'nu_s' : (0.0, 0.035)}

sobol_indices = {}
sobol_indices_s = {}

for parameter, chain in chains.items():
    # Quadrature points
    quadrature_points[parameter], delta_points[parameter] = np.linspace(
        limits[parameter][0], limits[parameter][1], n_quadrature_points[parameter], retstep=True
    )

    # Quadrature weights
    temp = np.ones((n_quadrature_points[parameter]))
    temp[0] = 0.5
    temp[-1] = 0.5
    quadrature_weights[parameter] = temp * delta_points[parameter]

    # Densities
    numerical_pdfs[parameter] = kdes[parameter](quadrature_points[parameter])
    
    # Quad * densitiies
    quadrature_pdfs[parameter] = quadrature_weights[parameter] * numerical_pdfs[parameter]
    

quadrature_mesh = np.meshgrid(*quadrature_points.values())
# quadrature_weights_mesh = np.meshgrid(*quadrature_weights.values())


def tensorial_product(x):
    return (
        x["c_1"].reshape(-1, 1, 1)
        * x["V_wall"].reshape(1, -1, 1)
        * x["nu_s"].reshape(1, 1, -1)
    )


# quadrature_weights_mesh = tensorial_product(quadrature_weights)
# joint_pdf_mesh = tensorial_product(numerical_pdfs)
# weights = quadrature_weights[k] * numerical_pdfs[k]

# sum is close to 1
quad_pdf_mesh = tensorial_product(quadrature_pdfs) # quadrature_weights_mesh * joint_pdf_mesh

In [9]:
# Sanity checks
for k in parameter_keys:
    print(f"int f for {k} is : ", np.sum(quadrature_pdfs[k]))
print("Joint integral is ", np.sum(quad_pdf_mesh))
# np.sum(np.sum(joint_pdf_mesh * quadrature_weights_mesh, axis=1), axis=1)
# np.sum(numerical_pdfs["nu_s"] * quadrature_weights["nu_s"])
# quad_pdf_mesh.shape

int f for c_1 is :  0.9986592673203363
int f for V_wall is :  1.0012719140247637
int f for nu_s is :  0.9999954089813299
Joint integral is  0.9999248853535075


## Evaluate model at these quadrature points

In [10]:
n_points = quad_pdf_mesh.size
f_evals_across_stations = -np.inf * np.ones(quad_pdf_mesh.shape + (len(y_stations),))

for i, j, k in tqdm(product(*map(range, n_quadrature_points.values()))):
    eval_params = (
        quadrature_points["c_1"][i],
        quadrature_points["V_wall"][j],
        quadrature_points["nu_s"][k],
    )
    f_evals_across_stations[i, j, k] = signal_from_(eval_params)

In [11]:
print("Please select the QoI index using the variable below")
signal_index = 3

# Take the first index
f_evals = f_evals_across_stations[..., signal_index]

Please select the QoI index using the variable below


## Zero-order
### Calculate $f_0$ and $D$

In [12]:
sobol_indices["f_0"] = np.sum(quad_pdf_mesh * f_evals)
sobol_indices["total_variance"] = (
    np.sum(quad_pdf_mesh * (f_evals ** 2)) - (sobol_indices["f_0"] ** 2)
)

print("f_0 : ", sobol_indices["f_0"])
print("D : ", sobol_indices["total_variance"])

f_0 :  0.5059037367466327
D :  0.00037337837188144984


## First-order
### Calculate $f_{i}$

In [13]:
def key2ind(key):
    return parameter_keys.index(key)

def calculate_first_order_integral_along_(key):
    ind = key2ind(key)

    def compare_with(another_key):
        if key == another_key:
            quad_pdf = np.ones_like(quadrature_weights[another_key])
        else:
            quad_pdf = quadrature_pdfs[another_key]
        return quad_pdf

    quad_pdf_c1 = compare_with("c_1")
    quad_pdf_v = compare_with("V_wall")
    quad_pdf_n = compare_with("nu_s")

    quad_local_pdfs = (
        quad_pdf_c1.reshape(-1, 1, 1)
        * quad_pdf_v.reshape(1, -1, 1)
        * quad_pdf_n.reshape(1, 1, -1)
    )
    ax = [0, 1, 2]
    ax.remove(ind)
    return np.sum(quad_local_pdfs * f_evals, axis=tuple(ax))


def calculate_first_order_sobol_along(key, indices):
    return calculate_first_order_integral_along_(k) - indices["f_0"]

for k in parameter_keys:
    sobol_indices[k] = calculate_first_order_sobol_along(k, sobol_indices)

### Calculate $D_i$

In [14]:
for k in parameter_keys:
    sobol_indices["variance_" + k] = np.sum(quadrature_pdfs[k] * (sobol_indices[k] ** 2))

In [15]:
for k in parameter_keys:
    print(f"Variance in {k} :", sobol_indices["variance_" + k])

Variance in c_1 : 0.00018902278122860636
Variance in V_wall : 0.00010422682235538212
Variance in nu_s : 5.472356075739316e-06


### Calculate $S_i$

In [16]:
for k in parameter_keys:
    sobol_indices_s[k] = sobol_indices["variance_" + k] / sobol_indices["total_variance"]
    print(f"First order sobol index in {k} :", sobol_indices_s[k])

First order sobol index in c_1 : 0.5062499476767293
First order sobol index in V_wall : 0.2791453126494291
First order sobol index in nu_s : 0.01465632850709635


## Second-order
### Calculate $f_{ij}$

In [17]:
def calculate_second_order_integral_along_(key1, key2):
    ind1 = key2ind(key1)
    ind2 = key2ind(key2)

    ax = [0, 1, 2]
    ax.remove(ind1)
    ax.remove(ind2)

    ind3 = ax[0]
    key = parameter_keys[ind3]

    def compare_with(another_key):
        if key != another_key:
            quad_pdf = np.ones_like(quadrature_weights[another_key])
        else:
            quad_pdf = quadrature_pdfs[another_key]
        return quad_pdf

    quad_pdf_c1 = compare_with("c_1")
    quad_pdf_v = compare_with("V_wall")
    quad_pdf_n = compare_with("nu_s")

    quad_local_pdfs = (
        quad_pdf_c1.reshape(-1, 1, 1)
        * quad_pdf_v.reshape(1, -1, 1)
        * quad_pdf_n.reshape(1, 1, -1)
    )
    return np.sum(quad_local_pdfs * f_evals, axis=tuple(ax))


def calculate_second_order_sobol_along(key1, key2, indices):
    return (
        calculate_second_order_integral_along_(key1, key2)
        - indices[key1].reshape(-1, 1)
        - indices[key2].reshape(1, -1)
        - indices["f_0"]
    )

sobol_indices["c_1,V_wall"] = calculate_second_order_sobol_along("c_1", "V_wall", sobol_indices)
sobol_indices["V_wall,nu_s"] = calculate_second_order_sobol_along("V_wall", "nu_s", sobol_indices)
sobol_indices["c_1,nu_s"] = calculate_second_order_sobol_along("c_1", "nu_s", sobol_indices)

### Calculate $D_{ij}$

In [18]:
parameter_pair_keys = ["c_1,V_wall", "V_wall,nu_s", "c_1,nu_s"]
for k in parameter_pair_keys:
    k1, k2 = k.split(",")
    pair_quad_pdfs = np.outer(quadrature_pdfs[k1], quadrature_pdfs[k2])
    sobol_indices["variance_" + k] = np.sum(pair_quad_pdfs * (sobol_indices[k] ** 2))

In [19]:
for k in parameter_pair_keys:
    print(f"Variance in {k} :", sobol_indices["variance_" + k])

Variance in c_1,V_wall : 7.697644882109902e-08
Variance in V_wall,nu_s : 2.2285278772066632e-09
Variance in c_1,nu_s : 5.637291972280213e-05


### Calculate $S_{ij}$

In [20]:
for k in parameter_pair_keys:
    sobol_indices_s[k] = sobol_indices["variance_" + k] / sobol_indices["total_variance"]
    print(
        "Second order sobol index for {} :".format(tuple(k.split(","))),
        sobol_indices_s[k]
    )

Second order sobol index for ('c_1', 'V_wall') : 0.00020616204530866496
Second order sobol index for ('V_wall', 'nu_s') : 5.968551059819383e-06
Second order sobol index for ('c_1', 'nu_s') : 0.1509806779614458


In [21]:
# Total sum of sobol indices
from functools import reduce
reduce(lambda x, y : x + y, sobol_indices_s.values())
# sobol_indices_s.values()

0.9512443973910689

In [22]:
for k, v in sobol_indices_s.items():
    print(k, v)

c_1 0.5062499476767293
V_wall 0.2791453126494291
nu_s 0.01465632850709635
c_1,V_wall 0.00020616204530866496
V_wall,nu_s 5.968551059819383e-06
c_1,nu_s 0.1509806779614458


## Total Sensitivity

In [23]:
sensitivities = {}
sensitivities["c_1"] = (
    sobol_indices_s["c_1"]
    + sobol_indices_s["c_1,V_wall"]
    + sobol_indices_s["c_1,nu_s"]
)
sensitivities["V_wall"] = (
    sobol_indices_s["V_wall"]
    + sobol_indices_s["c_1,V_wall"]
    + sobol_indices_s["V_wall,nu_s"]
)
sensitivities["nu_s"] = (
    sobol_indices_s["nu_s"]
    + sobol_indices_s["c_1,nu_s"]
    + sobol_indices_s["V_wall,nu_s"]
)

In [24]:
for k in parameter_keys:
    print(f"Total sensitivity to parameter {k} :", sensitivities[k])

Total sensitivity to parameter c_1 : 0.6574367876834837
Total sensitivity to parameter V_wall : 0.2793574432457976
Total sensitivity to parameter nu_s : 0.16564297501960198


## Sanity checks, ignore

In [25]:
# Sanity check for first order sobol index
index = 0
n_points = n_quadrature_points["V_wall"]
sobol = np.zeros((n_points, ))
for idx in range(n_points):
    another_key = "c_1"
    temp = quadrature_weights[another_key] * numerical_pdfs[another_key]
    another_key = "nu_s"
    another_temp = quadrature_weights[another_key] * numerical_pdfs[another_key]
    sobol[idx] = np.sum(f_evals[:, idx, :] * np.outer(temp, another_temp))
    
print(sobol - sobol_indices["f_0"])

[-0.05026221 -0.04519953 -0.04013685 -0.03507416 -0.03001148 -0.0249488
 -0.01988611 -0.01482343 -0.00976074 -0.00469806  0.00036462  0.00542731
  0.01048999  0.01555267  0.02061536  0.02567804  0.03074072  0.03580341
  0.04086609  0.04592877  0.05099146]


In [ ]:
# Sanity check for second order sobol indices
key1 = "c_1"
key2 = "nu_s"
local_params = parameter_keys.copy()
local_params.remove(key1)
local_params.remove(key2)
key3 = local_params[0]

n_points1 = n_quadrature_points[key1]
n_points2 = n_quadrature_points[key2]

sobol2 = np.zeros((n_points1, n_points2))

for i in range(n_points1):
    for j in range(n_points2):    
        temp = quadrature_pdfs[key3]
        s = (i, slice(None), j)
        sobol2[i, j] = np.sum(f_evals[s] * temp) - sobol_indices[key1][i] - sobol_indices[key2][j] - sobol_indices["f_0"]
sobol2